<a href="https://colab.research.google.com/github/sudarshbuxyyes/Hebbian_MetaRL/blob/main/FSL_World.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade setuptools==65.5.0
!pip install learn2learn
!pip install import_ipynb
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import numpy as np
import gym
from gym import spaces
import learn2learn as l2l
from learn2learn.data import *
import import_ipynb
import utils
import pandas as pd
from torch.utils.data import Dataset
import pickle
from queue import Queue
from threading import Thread
from numpy.distutils.misc_util import is_sequence
import threading
# from stable_baselines3 import PPO,DQN,A2C,SAC
import gym
from gym import spaces,Env

In [ ]:
class Controller():
    def __init__(self,parent=None):
        self.parent=parent
    def act(self,world_state):
        perceived_state=self.parent.perception.perceive_state(world_state)
        if self.parent.debug: print(perceived_state)
        if self.parent.use_memory: self.parent.memory.add_percept(perceived_state,self.parent.time)
        actor_state=self.parent.actor.percept_to_state(perceived_state)
        if self.parent.use_memory: actor_state=self.parent.actor.augment_state(actor_state)
        if self.parent.debug: print('actor_state:',actor_state)
        if self.parent.use_memory: self.parent.memory.update_next_state(actor_state,self.parent.time-1)
        action=self.parent.actor.call_model(actor_state)
        if self.parent.debug: print('action:',action)
        if self.parent.use_memory: action_to_store=self.parent.perception.action_perceptual(action)
        if self.parent.use_memory: 
            intrinsic_reward=self.parent.actor.intrinsic_reward(perceived_state,action_to_store,actor_state)
        # if self.parent.use_memory: self.parent.memory.update_reward_sar(intrinsic_reward,self.parent.time-1)
        if self.parent.use_memory: self.parent.memory.update_reward_sar(intrinsic_reward,self.parent.time)
        if self.parent.use_memory: self.parent.memory.update_action_perceptual(action_to_store,self.parent.time)
        if self.parent.use_memory: self.parent.memory.add_state_action(actor_state,action_to_store,self.parent.time)
        world_action=self.parent.perception.action_to_world(action)
        if self.parent.limit_memory: self.parent.memory.pop()
        return world_action
    def reward(self,world_reward):
        reward=self.parent.perception.perceive_reward(world_reward)
        if self.parent.use_memory: self.parent.memory.update_reward_perceptual(reward,self.parent.time-1)
        if self.parent.use_memory: reward_sar=self.parent.actor.compute_reward(reward)
        if self.parent.use_memory: self.parent.memory.update_reward_sar(reward_sar,self.parent.time-1)
        if self.parent.limit_memory: self.parent.memory.pop() 
        return reward


In [ ]:
class Memory():
    def __init__(self,parent=None):
        self.parent=parent
        self.clear()
        self.limit_perceptual=None
        self.limit_sar=None
    def pop(self):
        psize=self.limit_perceptual
        M=self.perceptual_memory
        if psize!=None: 
            for t in [t for t in M if t<self.parent.time-psize]: M.pop(t)
        S=self.sar_memory
        ssize=self.limit_sar
        if ssize!=None: 
            for t in [t for t in S if t<self.parent.time-ssize]: S.pop(t)
    def clear(self):
        self.perceptual_memory={}
        self.sar_memory={}
    def add_percept(self,perceived_state,time):
        self.perceptual_memory[time]=perceived_state
        if self.parent.debug: print('add_percept:',self.perceptual_memory)
    def update_next_state(self,actor_state,time):
        if time in self.sar_memory: self.sar_memory[time]['next_state']=actor_state
        else: self.sar_memory[time]={'next_state':actor_state}
        if self.parent.debug: print('update_next_state:',self.sar_memory)
    def update_action_perceptual(self,action,time):
        self.perceptual_memory[time]['action']=action
        if self.parent.debug: print('update_action_perceptual:',self.perceptual_memory)
    def add_state_action(self,actor_state,action,time):
        if time in self.sar_memory: 
            self.sar_memory[time]['state']=actor_state
            self.sar_memory[time]['action']=action
        else: self.sar_memory[time]={'state':actor_state,'action':action}
        if self.parent.debug: print('add_state_action:',self.sar_memory)
    def update_reward_perceptual(self,reward,time):
        self.perceptual_memory[time]['reward']=reward
        if self.parent.debug: print('update_reward_perceptual:',self.perceptual_memory)
    def update_reward_sar(self,reward,time):
        if time in self.sar_memory:
            if 'reward' in self.sar_memory[time]:self.sar_memory[time]['reward']+=reward
            else: self.sar_memory[time]['reward']=reward
        else:
            self.sar_memory[time]={'reward':reward}
        if self.parent.debug: print('update_reward_sar:',self.sar_memory)

In [ ]:
class Perception():
    def __init__(self,parent=None):
        self.parent=parent
    def perceive_state(self,world_state):
        if type(world_state)==dict: return world_state
        else: return {'percept':world_state}
    def action_to_world(self,action):
        return action
    def action_perceptual(self,action):
        return action
    def perceive_reward(self,reward):
        return reward

In [ ]:
class Actor():
    def __init__(self,parent=None,model=None):
        self.parent=parent
        self.model=model
        self.default_action='default_action'
        self.intrinsic_reward_value=0
    def percept_to_state(self,perceived_state):
        return perceived_state['percept']
    def augment_state(self,state):
        return state
    def call_model(self,actor_state):
        return self.default_action
    def compute_reward(self,reward):
        return reward
    def intrinsic_reward(self,precept,action,state):
        return self.intrinsic_reward_value

In [ ]:
class AIAgent():
    def __init__(self,controller=None,perception=None,memory=None,actor=None):
        if controller is not None: self.controller=controller
        else: self.controller=Controller(parent=self)
        if perception is not None: self.perception=perception
        else: self.perception=Perception(parent=self)
        if memory is not None: self.memory=memory
        else: self.memory=Memory(parent=self)
        if actor is not None: self.actor=actor
        else: self.actor=Actor(parent=self)
        self.time=0
        self.ep=[]
        # self.controller.parent=self
        # self.perception.parent=self
        # self.memory.parent=self
        # self.actor.parent=self
        self.debug=False
        self.use_memory=True
        self.limit_memory=False
    def act(self,world_state):
        world_action = self.controller.act(world_state)
        # check to see if network needs training - TBDesigned
        self.time+=1
        return world_action
    def reward(self,world_reward):
        return self.controller.reward(world_reward)
    def episode(self):
        return self.controller.episode()
    def begin(self,state=None):
        ## TBD may need to do more - both episode and time may be needed and episode reset
        self.ep+=[self.time]
    def clear(self):
        self.memory.clear()

In [ ]:
class KShotLoader():
    def __init__(self,myds,num_tasks=1000,shots=2,ways=2,classes=None):
        self.shots = shots
        self.ways = ways
        self.myMds = l2l.data.MetaDataset(myds)
        if classes == None:
            n_classes = len(set(myds.labels))
            classes = [i for i in range(n_classes)]
        self.my_tasks = l2l.data.TaskDataset(self.myMds, task_transforms=[
                                l2l.data.transforms.FilterLabels(self.myMds,classes),
                                l2l.data.transforms.NWays(self.myMds,ways),
                                l2l.data.transforms.KShots(self.myMds,2*shots),
                                l2l.data.transforms.LoadData(self.myMds),
                                l2l.data.transforms.RemapLabels(self.myMds),
                                l2l.data.transforms.ConsecutiveLabels(self.myMds)
                                ],num_tasks=num_tasks)
    def get_task(self):
        data,labels = self.my_tasks.sample()
        adaptation_indices = np.zeros(data.size(0), dtype=bool)
        adaptation_indices[np.arange(self.shots*self.ways) * 2] = True
        evaluation_indices = torch.from_numpy(~adaptation_indices)
        adaptation_indices = torch.from_numpy(adaptation_indices)
        adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
        evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
        d_train = (adaptation_data,adaptation_labels)
        d_test = (evaluation_data,evaluation_labels)
        return d_train, d_test

In [ ]:
class FSLWorld():
    def __init__(self,meta_train_ds,meta_test_ds,n_classes):
        self.meta_train_ds=meta_train_ds
        self.meta_test_ds=meta_test_ds
        self.action_space=spaces.Discrete(n_classes)
        self.obs_dim=self.meta_train_ds[0][0].shape[-1]
        high = np.inf*np.ones(self.obs_dim)
        low = -high
        self.observation_space=spaces.Box(high=high,low=low)
    def run(self,agent=None,n_episodes=10,n_tasks_train=25,n_tasks_test=50):
        if 'training' not in agent.__dict__: agent.training=True
        for episode in range(n_episodes):
            agent.begin()
            count=0
            for tid in range(n_tasks_train):
                tot_rew=0
                meta_train_kloader=KShotLoader(self.meta_train_ds,shots=5,ways=5)
                d_train,d_test=meta_train_kloader.get_task()
                train_ds = utils.MyDS(d_train[0],d_train[1])
                train_ds=[(s.unsqueeze(0),l.unsqueeze(0)) for s,l in train_ds]
                for sample,label in train_ds:
                    count+=1
                    done=(count==len(train_ds))
                    action=agent.act(sample)
                    reward=(self.accuracy(action,label),{'label':label})
                    agent.reward((reward[0],done,reward[1]))
                    tot_rew+=reward[0]
                if 'end' in dir(agent): agent.end()
                print('episode,task: ',episode,tid,'avg reward: ',tot_rew/len(train_ds))
        agent.set_training(False)
        print('Training Over')
        agent.begin()
        for tid in range(n_tasks_test):
            test_rew=0
            meta_test_kloader=KShotLoader(self.meta_test_ds,shots=5,ways=5)
            d_train,d_test=meta_test_kloader.get_task()
            test_ds = utils.MyDS(d_test[0],d_test[1])
            test_ds=[(s.unsqueeze(0),l.unsqueeze(0)) for s,l in test_ds]
            for sample,label in test_ds:
                action=agent.act(sample)
                reward=(self.accuracy(action,label),{})
                agent.reward(reward)
                test_rew+=reward[0]
            print(f'Test {tid} Over; Accuracy: ',test_rew/len(test_ds))
    def accuracy(self,action,label):
        if (type(action)==np.ndarray): action=action[-1]
        if action==label[-1]: return 1
        else: return 0